## Using Perplexity (Sonar, Sonar Pro) and Gemini (1.5 Flash,1.5 Pro, 2.0 Flash) models to Automate web search and summarization

<b>Example used later in this notebook
* Company : F W Webb 
* Task : use web search to summarize MnA events involving the company of interest

<b> Success Criterions : 
* Relevance of item with company and task
* Assessbility and reliability of the source

<b>Prompts were designed and refined in multiple iterations using ChatGPT, Gemini and DeepSeek R1 models

### Import API keys

In [1]:
import yaml
with open("config.yml", "r") as file:
    api_keys = yaml.safe_load(file)

# print(api_keys)

### Prompt

In [2]:
from openai import OpenAI

perplexity_api_key = api_keys['perplexity_api_key']

messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                2. Only keep the results which mentions merger or acquisitions involving F W Webb. 
                3. Remove duplicate articles based on order of appearance in search results.
                4. Summarize each event in ≤2 lines, mentioning who merged/acquired whom and when, along with any key details.
                5. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY : Short summary, Web Link to Source:
                6. Focus only on verified and relevant sources.
                7. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [3]:
client = OpenAI(api_key=perplexity_api_key, base_url="https://api.perplexity.ai")

# chat completion without streaming
response = client.chat.completions.create(
    model="sonar",
    messages=messages,
)
print(response)

ChatCompletion(id='5e849b7e-5c5c-4728-b922-ea6cfc235054', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. **Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023**: F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply, both based in Pennsylvania, expanding its presence in the Philadelphia area. [5]\n2. **Danbury Plumbing and HVAC Supply Co. – Mar-2023**: F.W. Webb acquired Danbury Plumbing and HVAC Supply Co., enhancing its presence in Connecticut by adding another wholesale location and a Frank Webb Home showroom. [1][3]\n3. No additional acquisitions were found within the specified timeframe.', role='assistant', function_call=None, tool_calls=None), delta={'role': 'assistant', 'content': ''})], created=1738160223, model='sonar', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=114, prompt_tokens=249, total_tokens=363), citations=['h

#### Sonar API Output

In [4]:
response_dict = response.to_dict()
print(response_dict['choices'][0]['message']['content'])

1. **Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023**: F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply, both based in Pennsylvania, expanding its presence in the Philadelphia area. [5]
2. **Danbury Plumbing and HVAC Supply Co. – Mar-2023**: F.W. Webb acquired Danbury Plumbing and HVAC Supply Co., enhancing its presence in Connecticut by adding another wholesale location and a Frank Webb Home showroom. [1][3]
3. No additional acquisitions were found within the specified timeframe.


#### Sonar API Citations

In [5]:
for i,x in enumerate(response_dict['citations']):
    print(f"{i+1}: {x}")
# print(response_dict['citations'])

1: https://hvacinsider.com/f-w-webb-company-acquires-danbury-plumbing-and-hvac-supply-co/
2: https://edisciplinas.usp.br/pluginfile.php/5547326/mod_resource/content/1/Willmer,%20Stone%20%20Johsnton_Environmental-physiology-of-animals.pdf
3: https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/
4: https://www.ahajournals.org/doi/10.1161/CIR.0000000000000923
5: https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/


### Perplexity Sonar Pro API

In [6]:
response_pro = client.chat.completions.create(
    model="sonar-pro",
    messages=messages,
)
print(response_pro)

ChatCompletion(id='928a8065-58bd-4859-92c0-46aac862d5ad', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Danbury Plumbing and HVAC Supply Co. - Mar-2023: F.W. Webb acquired Western Connecticut wholesale distributor, increasing presence in Connecticut. https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/\n\n2. Rising Sun Plumbing Supply and Steinberg Plumbing Supply - Jun-2023: F.W. Webb acquired two Pennsylvania-based plumbing, heating and industrial supply distributors. https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/\n\n3. J.D. Johnson Co. - Dec-2023: F.W. Webb acquired J.D. Johnson Co. in Poughkeepsie, NY, expanding its presence in New York state. https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J.D.-Johnson-Co.\n\n4. Grant Supply Co. - Jun

In [39]:
response_dict = response.to_dict()
print(response_dict['choices'][0]['message']['content'])

1. **J.D. Johnson Co.** – Dec-2023 : F.W. Webb acquired J.D. Johnson Co., expanding its presence in Poughkeepsie, NY, with a new wholesale location offering extensive inventory and services. [5]
2. **Rising Sun Plumbing Supply and Steinberg Plumbing Supply** – Jun-2023 : F.W. Webb acquired these two Philadelphia-based distributors, enhancing its presence in the Greater Philadelphia market. [3]
3. **Danbury Plumbing and HVAC Supply Co.** – Mar-2023 : F.W. Webb acquired Danbury Plumbing, increasing its presence in Connecticut and expanding its services to more contractors. [1][2]
4. **Grant Supply Co.** – Jun-2015 : F.W. Webb acquired Grant Supply, adding locations in Pleasantville and North Brunswick, NJ, to its network. [4]


In [40]:
for i,x in enumerate(response_dict['citations']):
    print(f"{i+1}: {x}")
# print(response_dict['citations'])

1: https://hvacinsider.com/f-w-webb-company-acquires-danbury-plumbing-and-hvac-supply-co/
2: https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/
3: https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/
4: https://www.supplyht.com/articles/98609-fw-webb-acquires-grant-supply
5: https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J.D.-Johnson-Co.


#### Sonar Pro API Output

In [7]:
response_pro_dict = response_pro.to_dict()
print(response_pro_dict['choices'][0]['message']['content'])

1. Danbury Plumbing and HVAC Supply Co. - Mar-2023: F.W. Webb acquired Western Connecticut wholesale distributor, increasing presence in Connecticut. https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/

2. Rising Sun Plumbing Supply and Steinberg Plumbing Supply - Jun-2023: F.W. Webb acquired two Pennsylvania-based plumbing, heating and industrial supply distributors. https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/

3. J.D. Johnson Co. - Dec-2023: F.W. Webb acquired J.D. Johnson Co. in Poughkeepsie, NY, expanding its presence in New York state. https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J.D.-Johnson-Co.

4. Grant Supply Co. - Jun-2015: F.W. Webb acquired Grant Supply Co., adding two customer sites in southern New Jersey. https://www.supplyht.com/articles/98609-fw-webb-acquires-grant-supply


#### Sonar Pro API Citations

In [8]:
for i,x in enumerate(response_pro_dict['citations']):
    print(f"{i+1}: {x}")
# print(response_dict['citations'])

1: https://hvacinsider.com/f-w-webb-company-acquires-danbury-plumbing-and-hvac-supply-co/
2: https://www.mdm.com/news/tech-operations/operations/f-w-webb-acquires-danbury-plumbing-and-hvac-supply/
3: https://www.mdm.com/news/operations/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/
4: https://www.supplyht.com/articles/98609-fw-webb-acquires-grant-supply
5: https://www.businesswire.com/news/home/20231204042654/en/F.W.-Webb-Expands-to-Poughkeepsie-NY-Through-Its-Acquisition-of-J.D.-Johnson-Co.
6: https://www.inddist.com/home/news/13767943/fw-webb-now-a-master-distributor-for-armstrong-international
7: https://www.inddist.com/home/news/13769388/fw-webb-company-expands-into-pennsylvania


### Gemini 1.5 Flash

In [9]:
import google.generativeai as genai
gemini_api_key = api_keys['gemini_api_key']
genai.configure(api_key=gemini_api_key)

In [10]:
model_15flash = genai.GenerativeModel(
    "models/gemini-1.5-flash", system_instruction=messages[0]['content'], tools='google_search_retrieval'
)
response_15flash = model_15flash.generate_content(messages[1]['content'])
print(response_15flash.text)

1. J.D. Johnson Co. – Dec-2023: F.W. Webb acquired J.D. Johnson Co., opening a new wholesale location in Poughkeepsie, NY. This acquisition expanded F.W. Webb's presence in the Northeast.

2. Rising Sun Plumbing Supply & Steinberg Plumbing Supply – Jun-2023: F.W. Webb acquired two Pennsylvania-based plumbing distributors, Rising Sun and Steinberg, expanding its market reach in the Philadelphia area.

3. Danbury Plumbing and HVAC Supply Co. – Mar-2021: F.W. Webb acquired Danbury Plumbing and HVAC Supply Co., a Western Connecticut distributor serving Fairfield, Litchfield, and New Haven counties for over 100 years.




#### Geminin 1.5 Flash citations

In [11]:
response_15flash.candidates[0].grounding_metadata.grounding_chunks

[web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYvYpke4EfjqhCSYSeBtu2F_e2ZpVO0EtVHlQAPsqNRaD4xX0qK9Wh6moZNcYVSU9UbsLgLPIwu7uPRSdL_E7kfYjf5Gb7jRsudCMqNy7e710ye20sUHpEC1ZjDJJ1398dQ9hzAJojzj2CO7"
  title: "pitchbook.com"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYvdsRj6qBRhilOcIVkwJkfnL7yGPyiJ00ipCO1Wmk_AIYlZPhpWRdbqZWFQ9L9Tq7q3Tgr4BeGKb7vL7cbTBsOjFPtftOcT-HqS9DzqvJ8Ltq24cx-y0c0fmRGKjvTfGTHUo9bUNZXM0oi7YaR63iwGM_7JH-ZpNsvBttdNL8HMMEAl"
  title: "citybiz.co"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtt2pDxzqqq_l35Qpz_YMR12cPYiezlUN3w6O4PvmNjjmj2j0XX2__ZKl7pFQNwN77hiZinXPRX-EM3JjldVoKM9GXBE6S_SR4TjFlvJwYVGCeY6MQSJxPGWD0G79xRa83uWq7hmQF_He1mWzQuFCY_fKbd5jp31679vp1S2U-bYBz0"
  title: "supplyht.com"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYvpNrZ8i40BlDpQwgRO52Dqrtnd6Zah4mqJ2Gvh0_sSC70tzGB3XS8aZ3XZJN3NybDiMXBStD_vay4mA5n7ThprKM

### Gemini 1.5 Pro

In [12]:
model_15pro = genai.GenerativeModel(
    "models/gemini-1.5-pro", system_instruction=messages[0]['content'], tools='google_search_retrieval'
)
response_15pro = model_15pro.generate_content(messages[1]['content'])
print(response_15pro.text)

1. J.D. Johnson Co. – Dec-2023: F.W. Webb acquired J.D. Johnson Co., opening a new wholesale location in Poughkeepsie, NY.
2. Rising Sun Plumbing Supply & Steinberg Plumbing Supply – Jun-2023: F.W. Webb acquired these two Pennsylvania-based plumbing distributors.
3. Danbury Plumbing and HVAC Supply Co. – Mar-2021: F.W. Webb acquired this Western Connecticut wholesale distributor.





#### Geminin 1.5 Pro Citations

In [13]:
response_15pro.candidates[0].grounding_metadata.grounding_chunks

[web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYuVDi0-A_wlCfuAPfd4qh7MvHdSV7782uC8XDMY10ysU6sPXAmxaWwP_HXthRQdh2NZS_874_Ji6CWp19cbx6ay12x6IfBFHS3KdK7V3FmajR8Nbbnx5d8GBycsFFooSgVRo1zEGwlofCvK"
  title: "pitchbook.com"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYvYTAFHKpLlihnBSSeW4sER5c3Pk2xAqIFy9JaCS2PkxnMTz6HlOiIp6vb3Pyu_nJ4jBUFJ5fu-HVuvAEpFfnMgAkGBKJCS3mw5Y5f6GaNKsj90KmM0eeDCyuy6JrZczO0ZAca6jabVDeAfU9CmH2ujNSB6sDNhZ8Fi9KHvj8CSbPEB"
  title: "citybiz.co"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYubV5ZPc53-DLeQwELWxbGGVSZ1AgvERHjQgm7nCA7OvVnp4RNxqNaDCN6ebRuHVDSL3d5dcvjFmagl8zeDcmySyCoFZlkTIX624H5DC8fiS38kJc_01kxfW4Gdo3eOI7h6N8AJVJxk04S7PSbj7mnixUD1iJJC4T_CMvHqrq4LfmDo"
  title: "supplyht.com"
}
, web {
  uri: "https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYv8rHNIZkEei8MtCILr7Gq5cRpcj8VHwEIsidAG8253EEQpIudk_61zjs2a545MHGOCvkqMmAraCEaWzkCGydF9yu

In [68]:
# !pip install -U -q google-genai

### Gemini 2.0 Flash - Latest Model from Gemini Family

In [14]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


E:\Anaconda\lib\site-packages\pydantic\_internal\_generate_schema.py:355: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [15]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023: F.W. Webb acquired J.D. Johnson Co., expanding its presence in Poughkeepsie, New York. This acquisition added to F.W. Webb's 150+ year commitment to customer service. Web Link to Source: [https://supplyht.com/articles/103007-fw-webb-acquires-jd-johnson-co](https://supplyht.com/articles/103007-fw-webb-acquires-jd-johnson-co)
2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023: F.W. Webb acquired these two Pennsylvania-based plumbing suppliers, expanding its reach in the Philadelphia metro area. Both companies have served the region for decades. Web Link to Source: [https://www.inddist.com/mergers-acquisitions/news/22528278/fw-webb-acquires-pennsylvania-plumbing-suppliers](https://www.inddist.com/mergers-acquisitions/news/22528278/fw-webb-acquires-pennsylvania-plumbing-suppliers)
3.  VJ Stanley and Stritt & Priebe – Jan-2022: F.W. Webb acquired both VJ Stanley and Stritt & Priebe. The specific details of these acquisitions were not dis

#### Gemini 2.0 Citations

In [16]:
print(response_20flash.candidates[0].grounding_metadata.grounding_chunks)

[GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='pitchbook.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYuD2Cr0vyc0NZTcGUeHsqT2eTxwli3YKiAikwcXvvNTCbpc_R_nIduKjEtOAjiDu0KYkslE9yXq5U-g8j62poAFeSDDfLdEDWDDq4kuWGsIChY19K-QblQJQjUhiKNbTS2CSbuvA1chJGSf')), GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='tracxn.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYu411X5nstzcSwKz9l_AaZaaI3tRBN_E154wt318bg7CGyUtTn0nGmBN2ZilUnblpILRRghNW8mseCWvf2DroJ0HJO7aVmES8Ui6OrZyPSYANDuRMfFJYZuXAeT7KYgyhYZlr5mSrIr_3zY3Kjqtf-W_bg2-ZERwJbnHm7S0Ousijbk5jXDj9f19LmNN_Nvhz3tKrNokXMuY-wLJBPv3uqsB5AT')), GroundingChunk(retrieved_context=None, web=GroundingChunkWeb(title='supplyht.com', uri='https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYuL2TwbI9nOgEfT1hXFkpETPWiChMd36CAV9RVxyVWWAFco9vPTvIGLcnoxe8iLaqAFo6VN1-zWOhIPFAC070_2GLfJAZFRKwuR3E09CUvbAjKs8yjBNkZqACbibbsNrmYopHVnMu9ar9oDU1wUAc_U77VQ5Drel

### Few Links doesnt seems appropriate and working. 
* <b> We observed that the citations links are routed through Vertex AI links and are not direct links in few cases
* <b> Few links provided in the main output dont work always. Can handled automatically through python)

### Prompt Variation 1

In [17]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Prioritize results where more details about the event are known like the nature of acquisition, date, benefits etc.
                4. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                5. Only keep the results which mentions merger or acquisitions involving F W Webb. 
                6. Remove duplicate articles based on order of appearance in search results.
                7. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Web address to this item from search results
                8. Focus only on verified and relevant sources.
                9. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [18]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [19]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023
    F.W. Webb acquired J.D. Johnson Co., a business serving the community for over 150 years, to expand its presence in Poughkeepsie, New York. The acquisition complements F.W. Webb's commitment to customer support.
    https://supplyht.com/articles/103277-fw-webb-acquires-jd-johnson-co

2.  Rising Sun Plumbing Supply – Jun-2023
    F.W. Webb acquired Rising Sun Plumbing Supply, a plumbing, heating, and industrial supply distributor in Philadelphia, to expand its reach in the Greater Philadelphia market.
    https://www.inddist.com/mergers-acquisitions/news/22529778/fw-webb-acquires-pennsylvania-plumbing-suppliers

3.  Steinberg Plumbing Supply – Jun-2023
    F.W. Webb acquired Steinberg Plumbing Supply, a plumbing, heating, and industrial supply distributor in Elkins Park, PA, to expand its reach in the Greater Philadelphia market.
    https://www.inddist.com/mergers-acquisitions/news/22529778/fw-webb-acquires-pennsylvania-plumbing-suppliers

4.  VJ Sta

### Prompt Approach 2

In [20]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                4. Only keep the results which mentions merger or acquisitions involving F W Webb.
                5. Remove duplicate articles based on order of appearance in search results. Only leveraged this filtered list of items further.
                6. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Web address to this item from search results
                7. Focus only on verified and relevant sources.
                8. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [21]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [22]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023
F.W. Webb acquired J.D. Johnson Co., a plumbing and HVAC supplier in Poughkeepsie, NY, expanding their presence in the New York market. This acquisition was completed in early December 2023.
[https://supplyht.com/articles/f-w-webb-acquires-j-d-johnson-co/](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYs7TtwgIa8F1fCDRA_xtUM7BdTtEuY9yXBxq4WrJ6pUj3Om4fYUGMxnCe3N0ItJML8saPQjKcJ11U18IV3wPFbLm4sJ0AUTChNQNrDxFbjCG4UK4NDy1AXS4vn5IC-pO_uiK5z7t49Fh8T9rKHc92ZsWLEpx-WNVWya-qKfaHwwhPE=)

2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023
F.W. Webb acquired two plumbing distributors in Pennsylvania, Rising Sun Plumbing Supply and Steinberg Plumbing Supply, expanding their reach in the Philadelphia area. The acquisition was announced in June 2023.
[https://www.mdm.com/news/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYsaKiV

### Prompt Approach 3

In [23]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                4. Only keep the results which mentions merger or acquisitions involving F W Webb.
                5. Remove duplicate articles based on order of appearance in search results. Only leveraged this filtered list of items further.
                6. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Public Web address to this item
                7. Focus only on verified and relevant sources.
                8. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [24]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [25]:
print(response_20flash.text)

1.  J.D. Johnson Co. – DEC-2023
    F.W. Webb acquired J.D. Johnson Co., a business serving the community for over 150 years, to expand its presence in the Poughkeepsie, New York area. The acquisition was completed on December 4, 2023, and the location became F.W. Webb's newest wholesale location.
    [https://supplyht.com/articles/102088-fw-webb-acquires-jd-johnson-co](https://supplyht.com/articles/102088-fw-webb-acquires-jd-johnson-co)

2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – JUN-2023
    F.W. Webb acquired Rising Sun Plumbing Supply and Steinberg Plumbing Supply, both located in the Philadelphia area, to expand its reach in the region. The acquisition was announced on June 5, 2023, and both companies now operate under the F.W. Webb umbrella.
    [https://www.mdm.com/news/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/](https://www.mdm.com/news/mergers-acquisitions/f-w-webb-acquires-2-plumbing-distributors-in-philadelphia/)

3.  

### Prompt Approach 4

In [26]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" and follow these steps:

                1. Scan the first 50 search results.
                2. Only keep publicly accessible resources.Avoid paid or proprietary databases where full data or direct access to the link is restricted.
                3. Keep the results where F W Webb is mentioned and the source has a published date. Only keep information going back 10 years only.
                4. Only keep the results which mentions merger or acquisitions involving F W Webb.
                5. Remove duplicate articles based on order of appearance in search results. Only leveraged this filtered list of items further.
                6. Create a numbered list (max 5 items) sorted by date (latest to oldest), formatted as:
                Company Acquired/Merged – MMM-YYYY
                A ≤2 lines summary , mentioning who merged/acquired whom and when, along with any key details.
                Link to this item from the original source
                7. Focus only on verified and relevant sources.
                8. Final response should only contains the numbered list with no additional text. If no eligible searches were found reponse only with the word No
                '''
        ),
    },
]


In [27]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [28]:
print(response_20flash.text)

1.  J D Johnson Co. – Dec-2023
    F.W. Webb acquired J.D. Johnson Co., a commercial and residential building trades supplier in Poughkeepsie, NY. This acquisition allowed F.W. Webb to expand its presence in the Dutchess County community.
    [https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtfUuUsmIoaE33f4I6SEie0IvPUDrSO30wZD40Z833PEVMtiCkSCMv1uUL_eZLCBwcJ_gZU_Y6-D0G315Q855ziNC2mkwKHp_20-J7HmLV-Url6JLphyw5ko9TDqr5xFwDnHR8l1z04cm_dT7UgSvTYYngPRVqUBsAmuKkFBDKaRX8B](https://vertexaisearch.cloud.google.com/grounding-api-redirect/AUBnsYtfUuUsmIoaE33f4I6SEie0IvPUDrSO30wZD40Z833PEVMtiCkSCMv1uUL_eZLCBwcJ_gZU_Y6-D0G315Q855ziNC2mkwKHp_20-J7HmLV-Url6JLphyw5ko9TDqr5xFwDnHR8l1z04cm_dT7UgSvTYYngPRVqUBsAmuKkFBDKaRX8B)
2.  Rising Sun Plumbing Supply and Steinberg Plumbing Supply – Jun-2023
    F.W. Webb acquired two plumbing distributors in Pennsylvania, Rising Sun Plumbing Supply and Steinberg Plumbing Supply, expanding its reach in the Philadelphia area. Both companies will oper

### Prompt Approach 5

In [29]:
messages = [
    {
        "role": "system",
        "content": (
            '''You are a market research analyst specializing in corporate mergers and acquisitions.
                Your task is to gather and summarize credible information from web searches. 
                You only consider trustworthy sources with clear credentials and safety certificates.
            '''
        ),
    },
    {   
        "role": "user",
        "content": (
            '''Search for "F W Webb merger acquisition" using Google and follow these steps:

        1. Scan the first 50 search results.

        2. Filter sources:

            2.1 Keep only publicly accessible sources (e.g., news articles, press releases, company websites).

            2.2 Exclude paid/proprietary databases or sources where full access is restricted.

        3. Retain relevant results:

            3.1 Include only results where F W Webb is explicitly mentioned in the context of mergers or acquisitions.

            3.2 Ensure the source has a published date within the last 10 years (based on the publication date, not the event date).

        4. Remove duplicates:

            4.1 Eliminate duplicate articles, keeping only the first instance based on order of appearance in search results.

        5. Exclude incomplete results:

            5.1 Remove any result where the full, original web link is not available or leads to a broken page.

        6. Verify relevance:

            6.1 Exclude results where F W Webb is mentioned in a context unrelated to mergers or acquisitions (e.g., as a partner, supplier, or in unrelated news).

        7. Sort the results by date (latest to oldest).
        8. Create a numbered list (max 5 items).Use the following format for each entry:

        8.1 Company Acquired/Merged – MMM-YYYY

        8.2 A summary in 2 lines or fewer, including:Who merged with or acquired whom.The date of the merger/acquisition.Any key details (e.g., financial terms, strategic rationale, or market impact).

        8.3 Provide the full, original web link to the article or webpage. Do not use shortened, tracking, or search result links.Focus on verified and reliable sources.Prioritize sources such as reputable news outlets, official press releases, or industry reports.

        9. If no eligible results are found.Return only "No" with no additional text.

                '''
        ),
    },
]


In [30]:
from google import genai
from google.genai.types import GenerateContentConfig, Tool

client = genai.Client(api_key=gemini_api_key)

config_20flash = GenerateContentConfig(system_instruction=messages[0]['content'], tools=[Tool(google_search={})], temperature=0)

response_20flash = client.models.generate_content(model="gemini-2.0-flash-exp", config=config_20flash, contents=messages[1]['content'])


In [31]:
print(response_20flash.text)

1.  J.D. Johnson Co. – Dec-2023
    F.W. Webb acquired J.D. Johnson Co., a business that had served the community for over 150 years. This acquisition allowed F.W. Webb to open a new wholesale location in Poughkeepsie, New York.
    [https://supplyht.com/articles/102008-fw-webb-acquires-jd-johnson-co](https://supplyht.com/articles/102008-fw-webb-acquires-jd-johnson-co)

2.  Rising Sun Plumbing Supply – Jun-2023
    F.W. Webb acquired Rising Sun Plumbing Supply, a plumbing, heating, and industrial supplier in Philadelphia. This acquisition expanded F.W. Webb's presence in the Greater Philadelphia market.
    [https://www.inddist.com/mergers-acquisitions/news/22537878/fw-webb-acquires-pennsylvania-plumbing-suppliers](https://www.inddist.com/mergers-acquisitions/news/22537878/fw-webb-acquires-pennsylvania-plumbing-suppliers)

3.  Steinberg Plumbing Supply – Jun-2023
    F.W. Webb acquired Steinberg Plumbing Supply, a plumbing, heating, and industrial supplier in Elkins Park, a suburb of P

### Geminis response on why web link extraction may not be completely reliable all the times

### Possible challenges in reliable web links extraction
* Search Result Snippets: 
LLMs often rely on the snippets provided by the search engine, which are short summaries of the web page content.  The actual URL might be associated with the entire page, but the LLM might only have access to the snippet and its associated (sometimes truncated or redirected) URL.

* Dynamic Content: 
Web pages are dynamic.  The content that the search engine indexed might be different from what's on the page when the LLM tries to access it.  This can lead to broken links or content mismatches.

* Paywalls and Restrictions: 
Even if the initial search result indicated a publicly accessible page, the website might have implemented changes (paywalls, registration requirements) that restrict access.  The LLM doesn't have a way to "log in" or bypass these restrictions.

* URL Shortening/Tracking: 
Some websites use URL shorteners or tracking links, which can make it difficult to determine the actual destination URL.  The LLM might present the shortened/tracked link, which could be less reliable.

* LLM Interpretation: 
Even with the correct URL, the LLM might not always present it correctly in the response.  Sometimes, it can get confused with formatting or other elements in the search result.

### Strategies to Improve Link Reliability:

* Explicitly Request Links: 
In your prompt, be very clear that you need the original, fully functional web links.  For example:  "Provide the full, original web link to each source.  Do not use shortened links or links to search result pages.  I need the direct link to the article or web page itself."  The more explicit you are, the better the chances.

* Limit Snippet Dependence (If Possible):  
If the API allows it, explore options to make the LLM rely less on snippets and more on the actual page content (though this might be more computationally expensive).  This is not always an available option but is worth investigating in the API documentation.

* Post-Processing:  
The most reliable approach is often to post-process the LLM's output.  This means taking the links provided by the LLM and then programmatically checking them for validity.  You can use a library like requests in Python to make HTTP requests to the URLs and check the status codes (200 OK, 404 Not Found, etc.).  This allows you to filter out broken links or redirect to paywalls.

* Specify Acceptable Domains:  
If you're looking for information from specific, reputable sources, you can tell the LLM to prioritize those domains.  This can increase the likelihood of getting reliable links.  For example, "Focus on sources from reputable news organizations like the New York Times, Reuters, or the Wall Street Journal."

* Iterative Refinement: 
If you get a broken link, try rephrasing your query or requesting the information again.  Sometimes, the LLM might provide a different link on a subsequent attempt.

* Human Verification: 
For critical information, always verify the links and the content yourself.  LLMs are powerful tools, but they are not perfect.  Human oversight is still essential.
